In [4]:
import numpy as np
from datasets import load_dataset
from time import time

In [5]:
# 1) Load and preprocess
def load_and_preprocess():
    """
    Load the CIFAR-10 dataset via Hugging Face Datasets, normalize and flatten it.

    This function downloads the CIFAR-10 train and test splits, scales all image
    pixel values from [0, 255] to [0.0, 1.0], and flattens each 32×32×3 image into
    a 3072-dimensional vector.

    Returns:
        X_train (np.ndarray): Training features, shape (50000, 3072), dtype float32.
        y_train (np.ndarray): Training labels, shape (50000,), dtype int64.
        X_test  (np.ndarray): Test features, shape (10000, 3072), dtype float32.
        y_test  (np.ndarray): Test labels, shape (10000,), dtype int64.
    """
    ds = load_dataset("cifar10")
    X_train = np.array(ds["train"]["img"], dtype=np.float32) / 255.0
    X_test  = np.array(ds["test"]["img"],  dtype=np.float32) / 255.0
    # X_train = np.array(ds["train"]["img"]) / 255.0  # shape (50000,32,32,3)
    y_train = np.array(ds["train"]["label"])
    # X_test  = np.array(ds["test"]["img"])  / 255.0
    y_test  = np.array(ds["test"]["label"])
    # flatten
    X_train = X_train.reshape(len(X_train), -1)
    X_test  = X_test.reshape(len(X_test),  -1)
    return X_train, y_train, X_test, y_test

In [12]:
# 3) Simple MLP
class MLP:
    """
    Multi-layer Perceptron classifier with ReLU hidden activations and softmax output.

    Parameters
    ----------
    layer_sizes : list of int
        Sizes of each layer, including input and output layers.
        E.g., [200, 256, 128, 10] means input dim 200 → hidden 256 → hidden 128 → output 10.
    lr : float, optional (default=1e-2)
        Learning rate for gradient descent updates.

    Attributes
    ----------
    weights : list of np.ndarray
        Weight matrices for each layer, initialized with Xavier uniform.
    biases : list of np.ndarray
        Bias vectors for each layer, initialized to zeros.

    Methods
    -------
    forward(X)
        Perform a forward pass through the network and return class probabilities.
    backward(y_true)
        Backpropagate the loss (softmax + cross-entropy) and update parameters.
    train(X, y, epochs=10, batch_size=128)
        Train the network on (X, y) using mini-batch gradient descent.
    predict(X)
        Compute class predictions for input data X.
    """
    def __init__(self, layer_sizes, lr=1e-2):
        self.lr = lr
        self.weights = []
        self.biases  = []
        # Xavier init
        for i in range(len(layer_sizes)-1):
            in_size, out_size = layer_sizes[i], layer_sizes[i+1]
            bound = np.sqrt(6/(in_size+out_size))
            W = np.random.uniform(-bound, bound, (in_size, out_size))
            b = np.zeros(out_size)
            self.weights.append(W)
            self.biases.append(b)

    def forward(self, X):
        self.activations = [X]
        for i in range(len(self.weights)-1):
            Z = self.activations[-1] @ self.weights[i] + self.biases[i]
            A = np.maximum(0, Z)  # ReLU
            self.activations.append(A)
        # last layer
        Z = self.activations[-1] @ self.weights[-1] + self.biases[-1]
        # softmax
        exps = np.exp(Z - Z.max(axis=1, keepdims=True))
        A = exps / exps.sum(axis=1, keepdims=True)
        self.activations.append(A)
        return A

    def backward(self, y_true):
        grads_W = []
        grads_b = []
        m = y_true.shape[0]
        # one-hot
        Y = np.zeros_like(self.activations[-1])
        Y[np.arange(m), y_true] = 1
        # gradient on softmax-crossentropy
        dA = (self.activations[-1] - Y) / m

        for i in reversed(range(len(self.weights))):
            A_prev = self.activations[i]
            dW = A_prev.T @ dA
            db = dA.sum(axis=0)
            grads_W.insert(0, dW)
            grads_b.insert(0, db)
            if i>0:
                dZ = dA @ self.weights[i].T
                dA = dZ * (A_prev > 0)  # ReLU backward

        # update
        for i in range(len(self.weights)):
            self.weights[i] -= self.lr * grads_W[i]
            self.biases[i]  -= self.lr * grads_b[i]

    def train(self, X, y, epochs=10, batch_size=128):
        n = X.shape[0]
        for ep in range(epochs):
            perm = np.random.permutation(n)
            X, y = X[perm], y[perm]
            t0 = time()
            for i in range(0, n, batch_size):
                xb = X[i:i+batch_size]
                yb = y[i:i+batch_size]
                self.forward(xb)
                self.backward(yb)
            # eval train loss/acc
            probs = self.forward(X[:1000])
            preds = np.argmax(probs, axis=1)
            acc = (preds == y[:1000]).mean()
            print(f"Epoch {ep+1}/{epochs}  train‐acc@1k: {acc*100:.2f}%  time: {time()-t0:.1f}s")

    def predict(self, X):
        probs = self.forward(X)
        return np.argmax(probs, axis=1)


In [2]:
X_train, y_train, X_test, y_test = load_and_preprocess()

README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/120M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

▶ Running PCA...


In [13]:
from sklearn.decomposition import PCA 
pca = PCA(n_components=200, svd_solver="auto", whiten=False, random_state=42)
pca.fit(X_train)
X_train_p = pca.transform(X_train)
X_test_p  = pca.transform(X_test)

In [14]:
print("▶ Training MLP on 200‐dim data...")
# architecture: 200 → 256 → 128 → 10
mlp = MLP([200, 256, 128, 10], lr=1e-2)
mlp.train(X_train_p, y_train, epochs=20, batch_size=256)


▶ Training MLP on 200‐dim data...
Epoch 1/20  train‐acc@1k: 28.10%  time: 0.4s
Epoch 2/20  train‐acc@1k: 33.60%  time: 0.5s
Epoch 3/20  train‐acc@1k: 36.00%  time: 0.5s
Epoch 4/20  train‐acc@1k: 38.60%  time: 0.6s
Epoch 5/20  train‐acc@1k: 39.20%  time: 0.5s
Epoch 6/20  train‐acc@1k: 42.20%  time: 0.5s
Epoch 7/20  train‐acc@1k: 42.90%  time: 0.5s
Epoch 8/20  train‐acc@1k: 40.10%  time: 0.6s
Epoch 9/20  train‐acc@1k: 45.50%  time: 0.6s
Epoch 10/20  train‐acc@1k: 43.40%  time: 0.6s
Epoch 11/20  train‐acc@1k: 43.10%  time: 0.6s
Epoch 12/20  train‐acc@1k: 43.50%  time: 0.6s
Epoch 13/20  train‐acc@1k: 43.00%  time: 0.6s
Epoch 14/20  train‐acc@1k: 43.30%  time: 0.6s
Epoch 15/20  train‐acc@1k: 44.50%  time: 0.6s
Epoch 16/20  train‐acc@1k: 45.40%  time: 0.6s
Epoch 17/20  train‐acc@1k: 47.40%  time: 0.5s
Epoch 18/20  train‐acc@1k: 47.20%  time: 0.5s
Epoch 19/20  train‐acc@1k: 47.70%  time: 0.5s
Epoch 20/20  train‐acc@1k: 47.80%  time: 0.6s


In [15]:
print("▶ Evaluating on test set...")
y_pred = mlp.predict(X_test_p)
test_acc = (y_pred == y_test).mean()
print(f"Test accuracy: {test_acc*100:.2f}%")

▶ Evaluating on test set...
Test accuracy: 46.98%


In [5]:
# 5) sample predictions
for i in range(5):
    xi = X_test_p[i:i+1]
    pred = mlp.predict(xi)[0]
    print(f"Sample {i}  true: {y_test[i]}  pred: {pred}")


Sample 0  true: 3  pred: 3
Sample 1  true: 8  pred: 8
Sample 2  true: 8  pred: 8
Sample 3  true: 0  pred: 8
Sample 4  true: 6  pred: 4


In [19]:
from sklearn.decomposition import PCA 
pca = PCA(n_components=300, svd_solver="auto", whiten=False, random_state=42)
pca.fit(X_train)
X_train_p = pca.transform(X_train)
X_test_p  = pca.transform(X_test)

In [25]:
X_train.shape

(50000, 3072)

In [2]:
import os

# tell OpenBLAS / MKL to use up to 8 threads
os.environ["OMP_NUM_THREADS"] = "8"
os.environ["MKL_NUM_THREADS"] = "8"
import numpy as np

class MLP:
    """
    Deep fully‐connected Perceptron network with:
      - ReLU activations
      - Softmax output & cross‐entropy loss with label smoothing
      - Batch normalization on each hidden layer
      - Dropout on hidden layers
      - L2 weight decay
      - Adam optimizer for all parameters

    Parameters
    ----------
    layer_sizes : list of int
        Sizes of each layer including input and output, e.g. [300, 512, 256, 128, 10].
    lr : float, default=1e-3
        Base learning rate for Adam.
    dropout_rate : float in [0,1), default=0.5
        Drop probability on hidden activations.
    weight_decay : float, default=1e-4
        L2 regularization coefficient.
    label_smoothing : float in [0,1), default=0.1
        ε for smoothing the one-hot targets: ŷ = (1–ε)·y + ε/K.
    beta1, beta2 : floats, default=(0.9, 0.999)
        Adam momentum hyperparameters.
    eps : float, default=1e-8
        Adam & batch‐norm stability constant.
    """
    def __init__(self, layer_sizes,
                 lr=1e-3,
                 dropout_rate=0.5,
                 weight_decay=1e-4,
                 label_smoothing=0.1,
                 beta1=0.9, beta2=0.999, eps=1e-8):
        self.layer_sizes = layer_sizes
        self.lr = lr
        self.dropout_rate = dropout_rate
        self.weight_decay = weight_decay
        self.label_smoothing = label_smoothing
        self.beta1, self.beta2, self.eps = beta1, beta2, eps
        self.t = 0  # time step for Adam

        L = len(layer_sizes) - 1  # number of layers
        # Parameters
        self.W = [None]*L
        self.b = [None]*L
   
        self.gamma = [None]*(L-1)
        self.beta = [None]*(L-1)
        # Adam moment estimates
        self.mW = []; self.vW = []
        self.mb = []; self.vb = []
        self.mg = []; self.vg = []
        self.mb_bn = []; self.vb_bn = []

        for i in range(L):
            in_dim, out_dim = layer_sizes[i], layer_sizes[i+1]
            bound = np.sqrt(6/(in_dim + out_dim))
             # create and cast here
            W_i = np.random.uniform(-bound, bound, (in_dim, out_dim)).astype(np.float32)
            b_i = np.zeros(out_dim, dtype=np.float32)

            self.W[i] = W_i
            self.b[i] = b_i
            
            self.mW.append(np.zeros_like(self.W[i]))
            self.vW.append(np.zeros_like(self.W[i]))
            self.mb.append(np.zeros_like(self.b[i]))
            self.vb.append(np.zeros_like(self.b[i]))
            if i < L-1:
                # batch-norm params for hidden layers
                self.gamma[i] = np.ones(out_dim)
                self.beta[i]  = np.zeros(out_dim)
                self.mg.append(np.zeros_like(self.gamma[i]))
                self.vg.append(np.zeros_like(self.gamma[i]))
                self.mb_bn.append(np.zeros_like(self.beta[i]))
                self.vb_bn.append(np.zeros_like(self.beta[i]))

    def forward(self, X, training=True):
        """
        Forward pass with batch‐norm + ReLU + dropout.
        Returns softmax probabilities.
        """
        self.cache = {'A':[X], 'bn':[]}
        A = X

        # Hidden layers
        for i in range(len(self.W)-1):
            Z = A @ self.W[i] + self.b[i]                        # linear
            mu = Z.mean(axis=0, keepdims=True)
            var = Z.var(axis=0, keepdims=True)
            inv_std = 1/np.sqrt(var + self.eps)
            Z_norm = (Z - mu)*inv_std
            # affine transform
            Z_tilde = self.gamma[i]*Z_norm + self.beta[i]
            A = np.maximum(0, Z_tilde)                           # ReLU
            # dropout
            if training and self.dropout_rate>0:
                mask = (np.random.rand(*A.shape) > self.dropout_rate)/(1-self.dropout_rate)
                A *= mask
            else:
                mask = None
            # store caches
            self.cache['A'].append(A)
            self.cache['bn'].append((Z, Z_norm, mu, inv_std, self.gamma[i], mask))

        # Output layer
        Z_out = A @ self.W[-1] + self.b[-1]
        exps = np.exp(Z_out - Z_out.max(axis=1, keepdims=True))
        probs = exps / exps.sum(axis=1, keepdims=True)
        self.cache['A'].append(probs)
        return probs

    def backward(self, y_true):
        """
        Backward pass through softmax‐CE (with label smoothing), then
        output layer, then hidden layers (dropout → ReLU → BN → linear).
        Updates all params via Adam.
        """
        m = y_true.shape[0]
        K = self.layer_sizes[-1]
        # make smoothed targets
        Y = np.zeros((m, K))
        Y[np.arange(m), y_true] = 1
        Y = (1 - self.label_smoothing)*Y + self.label_smoothing/K

        # dL/dZ_out
        probs = self.cache['A'][-1]
        dZ = (probs - Y) / m

        # gradients for output layer
        A_prev = self.cache['A'][-2]
        dW = A_prev.T @ dZ + self.weight_decay * self.W[-1]
        db = dZ.sum(axis=0)
        self._adam_update('W', -1, dW, db)

        # backprop into hidden layers
        dA = dZ @ self.W[-1].T
        for i in reversed(range(len(self.W)-1)):
            Z, Z_norm, mu, inv_std, gamma_i, mask = self.cache['bn'][i]
            A_prev = self.cache['A'][i]

            # dropout backward
            if mask is not None:
                dA *= mask
            # ReLU backward
            dZ_tilde = dA * (Z_norm * inv_std * 0 + 1)  # dummy to hint shape
            dZ_tilde = dA * (self.cache['A'][i+1] > 0)

            # BN backward
            N = Z.shape[0]
            dgamma = np.sum(dZ_tilde * Z_norm, axis=0)
            dbeta  = np.sum(dZ_tilde, axis=0)
            dZ_norm = dZ_tilde * gamma_i
            dvar = np.sum(dZ_norm * (Z - mu) * -0.5 * inv_std**3, axis=0, keepdims=True)
            dmu  = np.sum(dZ_norm * -inv_std, axis=0, keepdims=True) + \
                   dvar * np.mean(-2*(Z-mu), axis=0, keepdims=True)
            dZ_bn = dZ_norm * inv_std + dvar*2*(Z-mu)/N + dmu/N

            # linear layer grads
            dW = A_prev.T @ dZ_bn + self.weight_decay * self.W[i]
            db = dZ_bn.sum(axis=0)

            # Adam update for W,b,gamma,beta
            self._adam_update('W', i, dW, db)
            self._adam_update('BN', i, dgamma, dbeta)

            # propagate to next
            dA = dZ_bn @ self.W[i].T

    def _adam_update(self, kind, idx, grad_W, grad_b):
        """Helper to update either ('W',b) or ('BN',beta)."""
        self.t += 1
        if kind == 'W':
            # weights
            self.mW[idx] = self.beta1*self.mW[idx] + (1-self.beta1)*grad_W
            self.vW[idx] = self.beta2*self.vW[idx] + (1-self.beta2)*(grad_W**2)
            m_hat = self.mW[idx]/(1-self.beta1**self.t)
            v_hat = self.vW[idx]/(1-self.beta2**self.t)
            self.W[idx] -= self.lr * m_hat/(np.sqrt(v_hat)+self.eps)
            # biases
            self.mb[idx] = self.beta1*self.mb[idx] + (1-self.beta1)*grad_b
            self.vb[idx] = self.beta2*self.vb[idx] + (1-self.beta2)*(grad_b**2)
            m_hat_b = self.mb[idx]/(1-self.beta1**self.t)
            v_hat_b = self.vb[idx]/(1-self.beta2**self.t)
            self.b[idx]  -= self.lr * m_hat_b/(np.sqrt(v_hat_b)+self.eps)
        else:
            # batch-norm gamma & beta
            self.mg[idx] = self.beta1*self.mg[idx] + (1-self.beta1)*grad_W
            self.vg[idx] = self.beta2*self.vg[idx] + (1-self.beta2)*(grad_W**2)
            mg_hat = self.mg[idx]/(1-self.beta1**self.t)
            vg_hat = self.vg[idx]/(1-self.beta2**self.t)
            self.gamma[idx] -= self.lr * mg_hat/(np.sqrt(vg_hat)+self.eps)
            self.mb_bn[idx] = self.beta1*self.mb_bn[idx] + (1-self.beta1)*grad_b
            self.vb_bn[idx] = self.beta2*self.vb_bn[idx] + (1-self.beta2)*(grad_b**2)
            mbn_hat = self.mb_bn[idx]/(1-self.beta1**self.t)
            vbn_hat = self.vb_bn[idx]/(1-self.beta2**self.t)
            self.beta[idx]  -= self.lr * mbn_hat/(np.sqrt(vbn_hat)+self.eps)

    def train(self, X, y, epochs=100, batch_size=128):
        """Mini-batch training loop with periodic train-subset accuracy print."""
        n = X.shape[0]
        for ep in range(1, epochs+1):
            perm = np.random.permutation(n)
            Xs, ys = X[perm], y[perm]
            for i in range(0, n, batch_size):
                xb, yb = Xs[i:i+batch_size], ys[i:i+batch_size]
                self.forward(xb, training=True)
                self.backward(yb)
            # monitor
            probs = self.forward(X[:1000], training=False)
            preds = np.argmax(probs, axis=1)
            acc = (preds == y[:1000]).mean()*100
            print(f"Epoch {ep}/{epochs} — acc@1k: {acc:.2f}%")

    def predict(self, X):
        """Return predicted class indices for X."""
        probs = self.forward(X, training=False)
        return np.argmax(probs, axis=1)


In [6]:
X_train, y_train, X_test, y_test = load_and_preprocess()

In [7]:
from sklearn.decomposition import PCA 
pca = PCA(n_components=300, svd_solver="auto", whiten=False, random_state=42)
pca.fit(X_train)
X_train_p = pca.transform(X_train)
X_test_p  = pca.transform(X_test)

In [8]:
print("▶ Training MLP on 300‐dim data...")
# architecture: 200 → 256 → 128 → 10
mlp = MLP(
  layer_sizes=[300, 1024, 512, 256, 10],  # one extra wide hidden layer
  lr=3e-3,                                      # bump initial LR
  dropout_rate=0.3,                             # less aggressive dropout
  weight_decay=5e-5,                            # lighter L2
  label_smoothing=0.1
)


mlp.train(X_train_p, y_train, epochs=80, batch_size=128)

▶ Training MLP on 500‐dim data...
Epoch 1/80 — acc@1k: 52.30%
Epoch 2/80 — acc@1k: 59.10%
Epoch 3/80 — acc@1k: 61.30%
Epoch 4/80 — acc@1k: 67.20%
Epoch 5/80 — acc@1k: 67.50%
Epoch 6/80 — acc@1k: 72.60%
Epoch 7/80 — acc@1k: 74.10%
Epoch 8/80 — acc@1k: 73.70%
Epoch 9/80 — acc@1k: 76.30%
Epoch 10/80 — acc@1k: 77.40%
Epoch 11/80 — acc@1k: 80.90%
Epoch 12/80 — acc@1k: 79.70%
Epoch 13/80 — acc@1k: 83.90%
Epoch 14/80 — acc@1k: 83.40%
Epoch 15/80 — acc@1k: 84.60%
Epoch 16/80 — acc@1k: 83.60%
Epoch 17/80 — acc@1k: 85.80%
Epoch 18/80 — acc@1k: 87.20%
Epoch 19/80 — acc@1k: 87.20%
Epoch 20/80 — acc@1k: 87.10%
Epoch 21/80 — acc@1k: 88.90%
Epoch 22/80 — acc@1k: 88.60%
Epoch 23/80 — acc@1k: 90.60%
Epoch 24/80 — acc@1k: 91.00%
Epoch 25/80 — acc@1k: 89.80%
Epoch 26/80 — acc@1k: 90.40%
Epoch 27/80 — acc@1k: 92.30%
Epoch 28/80 — acc@1k: 91.40%
Epoch 29/80 — acc@1k: 90.90%
Epoch 30/80 — acc@1k: 91.50%
Epoch 31/80 — acc@1k: 91.70%
Epoch 32/80 — acc@1k: 90.80%
Epoch 33/80 — acc@1k: 92.70%
Epoch 34/80 — acc@

In [10]:
print("▶ Evaluating on test set...")
y_pred = mlp.predict(X_test_p)
test_acc = (y_pred == y_test).mean()
print(f"Test accuracy: {test_acc*100:.2f}%")

▶ Evaluating on test set...
Test accuracy: 58.56%
